In [1]:
import h5py
import os
import math

import py3Dmol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator

# The PDBbind dataset's structure

The main unzipped folder is called `/v2015`, and inside are contained one folder for each protein.

In [2]:
DATA_DIR = "../data"
PDBBIND_DIR = Path(DATA_DIR, "v2015")

protein_list = [f.name for f in PDBBIND_DIR.iterdir() if f.is_dir()]
print("number of folders inside: ")
print(len(protein_list))

number of folders inside: 
11918


### As we can see, there are 11918 folders in the dataset. 

Each of them contain four different files, and we will now see what they contain

In [3]:
# Inside the very first folder, there are these four files, one for each molecular representation
folder = Path(PDBBIND_DIR, protein_list[0])

files = []
filenames = []
for filename in folder.iterdir():
    print(filename)
    with open(filename, "r") as f: 
        content = f.read()
    files.append(content)
    filenames.append(filename)

../data/v2015/1q1m/1q1m_ligand.mol2
../data/v2015/1q1m/1q1m_ligand.sdf
../data/v2015/1q1m/1q1m_protein.pdb
../data/v2015/1q1m/1q1m_pocket.pdb


In [4]:
files[0]

'### \n### Created by X-TOOL on Fri Sep 26 17:34:23 2014\n### \n\n@<TRIPOS>MOLECULE\n1q1m_ligand\n   45    47     1     0     0\nSMALL\nGAST_HUCK\n\n\n@<TRIPOS>ATOM\n      1 C1         33.0460   31.0940   18.4330 C.ar      1 MOL         0.1411\n      2 C2         32.5060   30.4810   19.6090 C.ar      1 MOL         0.0522\n      3 C3         31.1140   30.6200   19.8430 C.ar      1 MOL        -0.0516\n      4 C4         30.2740   31.3340   18.9520 C.ar      1 MOL        -0.0437\n      5 C5         30.8390   31.9260   17.7920 C.ar      1 MOL        -0.0628\n      6 C6         32.2250   31.8120   17.5260 C.ar      1 MOL        -0.0307\n      7 C7         33.2940   29.7050   20.6230 C.2       1 MOL         0.1630\n      8 O12        32.6890   29.1670   21.7470 O.3       1 MOL        -0.1598\n      9 N13        33.7090   28.5200   22.4770 N.2       1 MOL        -0.1265\n     10 C14        34.8870   28.6610   21.8140 C.2       1 MOL         0.0884\n     11 C15        34.6190   29.4330   20.60

### The first file contains the structure of a ligand

In [5]:
view = py3Dmol.view(width=640, height=480)
view.addModel(files[0], "mol2")
view.setStyle({'stick': {'colorscheme': 'element'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### The second file is also about the ligand, however, it's giving us trouble, so we will skip it for now
We already know the ligand, anyway

In [6]:
mol = Chem.MolFromMolFile(filenames[1])

print(mol)

None


[11:20:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:20:29] Explicit valence for atom # 11 C, 5, is greater than permitted


### The third file contains the full structure of the protein

In [7]:
view = py3Dmol.view(width=640, height=480)
view.addModel(files[2])
view.setStyle({"model": -1}, {"cartoon": {'color':'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### The final file contains only the binding pocket

In [8]:
view = py3Dmol.view(width=640, height=480)
view.addModel(files[3])
view.setStyle({"model": -1}, {"cartoon": {'color':'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### The dataset also contains general information

Mainly, 11741 binding affinites are provided in *pKd*

Reminder :
*The dissociation constant, often expressed as pKd (where pKd = −log Kd), is a measure of the affinity between a protein and its ligand, with lower pKd values indicating higher affinity.*


In [9]:
def parse_index_file(filepath):
    data = []
    with open(filepath) as f:
        for line in f:
            if line.startswith('#'): 
                continue
            parts = line.strip().split()
            pdb_id = parts[0]
            affinity_str = parts[1].split('=')[-1]
            try:
                pKd = float(affinity_str)
            except:
                continue
            data.append({'PDB_ID': pdb_id, 'pKd': pKd})
    return pd.DataFrame(data)

GENERAL_INDEX_FILE = Path(PDBBIND_DIR, "INDEX_general_PL_data.2015")

df = parse_index_file(GENERAL_INDEX_FILE)

df

,PDB_ID,pKd
0,3zzf,2.20
1,3gww,2.46
2,1w8l,1.80
3,3fqa,2.35
4,1zsb,2.00
...,...,...
11737,7cpa,2.00
11738,2xuf,2.55
11739,1avd,2.70
11740,2xui,2.60


### In reality, this is split into two datasets : the core dataset, meant for testing, and the refined dataset, meant for training

In [10]:
# These are the two separate datasets that are meant for training and testing
CORE_INDEX_FILE = Path(PDBBIND_DIR, "INDEX_core_data.2013")
REFINED_INDEX_FILE = Path(PDBBIND_DIR, "INDEX_refined_data.2015")

try: 
    core = parse_index_file(CORE_INDEX_FILE)
    refined = parse_index_file(REFINED_INDEX_FILE)
except Exception as e:
    print("couldn't parse the two index files")
finally: 
    print("all index files corretly parsed")

all index files corretly parsed


### One thing to keep in mind is that most of our core dataset is already included in the refined dataset. For this reason, we must first do another filtering step to have two mutually exclusive datasets

In [11]:
print("samples in the core dataset: ")
print(core.shape[0])

print("samples in the refined dataset")
print(refined.shape[0])

print("samples in the core dataset that are also in the refined dataset")
# Illustrating how many of the samples in the core dataset are also present in the refined dataset
print(core[core["PDB_ID"].isin(refined["PDB_ID"])].shape[0])

# Our training dataset must be our refined dataset, MINUS the core dataset
train_df = refined[~refined["PDB_ID"].isin(core["PDB_ID"])] # Use ~ to negate : we only keep whats NOT in core

# Our testing dataset is simply what remains, the core dataset
test_df = core

print("new training dataset: \n ")
print(train_df.shape)
train_df.head()

samples in the core dataset: 
195
samples in the refined dataset
3706
samples in the core dataset that are also in the refined dataset
190
new training dataset: 
 
(3516, 2)


,PDB_ID,pKd
0,2r58,2.00
1,3c2f,2.35
2,3g2y,1.31
3,3pce,2.06
4,4qsu,1.90


### We would like to train a model that could predict the pKd from the structure of our ligand and our protein

- For that, we must encode the protein and ligand features into uniformly-sized vectors
- Our model will take in the two vectors, and output a single number
- Morgan fingerprints will be used as the first encoding for our molecules
- Later on, we will see what happens if we add ESM2 embeddings to enrich our data

### First, let's generate Morgan fingerprints and save our preprocessed train / test vectors

In [ ]:
# Iterate through all of the ligands included in our train and test datasets
train_ids = train_df["PDB_ID"].to_list()
test_ids = test_df["PDB_ID"].to_list()


def generate_fingerprints_for_ids(pdb_ids: list[str]):
    fpSize=2048
    # This list will contain all of the fingerprints
    fingerprints = []
    # This will contain the aligned vector of IDs retained after the filtering steps
    retained_ids = []
    for pdb_id in pdb_ids:
        # The mol2 file is stored as such, see above file structure
        ligand_mol2_path = Path(PDBBIND_DIR, pdb_id, pdb_id + "_ligand.mol2")
        # Make sure the file actually exists, otherwise skip
        if not os.path.exists(ligand_mol2_path):
            continue
        # If it does exist, load its content
        mol = Chem.MolFromMol2File(ligand_mol2_path, sanitize=False, removeHs=False)
        # Again, we skip it if the molecule didn't load
        if mol is None:
            continue

        # If the molecule was successfully loaded, sanitize it to prevent errors
        try:
            mol.UpdatePropertyCache(strict=False)
            sanitize_status = Chem.SanitizeMol(mol, catchErrors=True)
            if sanitize_status != Chem.SanitizeFlags.SANITIZE_NONE:
                # Sanitization failed, skip this molecule
                print(f"Skipping {pdb_id}: sanitization returned code {sanitize_status}")
                continue
            # Ensure ring information is initialized
            Chem.rdmolops.FastFindRings(mol)
        except Exception as e:
            print(f"Skipping {pdb_id}: sanitization error {e}")
            continue

        # Now compute Morgan fingerprint
        morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=fpSize)

        # Generate fingerprint
        fp = morgan_gen.GetFingerprint(mol)

        # Convert to numpy array
        fp_array = np.array(fp)
        fingerprints.append(fp_array)
        retained_ids.append(pdb_id)
    # Convert our python array to a numpy array
    fingerprints_array = np.stack(fingerprints)
    return fingerprints_array, retained_ids

# Generate fingerprints for our train and test data, keeping track of which IDs are retained
train_fingerprints, train_ids = generate_fingerprints_for_ids(train_ids)
test_fingerprints, test_ids = generate_fingerprints_for_ids(test_ids)

[11:20:32] 3mof_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[11:20:32] Explicit valence for atom # 6 N, 5, is greater than permitted


Skipping 4lhm: sanitization returned code SANITIZE_PROPERTIES


[11:20:32] 2rk8_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[11:20:33] Explicit valence for atom # 8 P, 6, is greater than permitted


Skipping 1wc1: sanitization returned code SANITIZE_PROPERTIES


[11:20:33] Can't kekulize mol.  Unkekulized atoms: 3 4 19 20 22
[11:20:33] 3t01_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.


...

Skipping 1mue: sanitization returned code SANITIZE_KEKULIZE


[11:20:44] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 5


Skipping 1ksn: sanitization returned code SANITIZE_KEKULIZE


[11:20:45] Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5


Skipping 1sl3: sanitization returned code SANITIZE_KEKULIZE


### As we can see, Morgan fingerprints are stored as vectors of 2048 dimensions, with 1-hot encoded molecular features.

In [13]:
train_fingerprints

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(3509, 2048))

In [14]:
# Filter the pKd tables so that we drop columns that weren't retained
train_df = train_df[train_df["PDB_ID"].isin(train_ids)]
test_df = test_df[test_df["PDB_ID"].isin(test_ids)]

# Create numpy arrays of our pKd values
train_pkds = train_df["pKd"].to_numpy()
test_pkds = test_df["pKd"].to_numpy()

In [15]:
# Save our preprocessed data to an interim data folder
INTERIM_PATH = Path(DATA_DIR, "interim")
os.makedirs(INTERIM_PATH, exist_ok=True)
OUTPUT_PATH = Path(INTERIM_PATH, "reg_preprocessed_1.npz")

np.savez_compressed(OUTPUT_PATH,
                    X_train=train_fingerprints,
                    X_test=test_fingerprints,
                    y_train=train_pkds,
                    y_test=test_pkds,
                    test_ids=test_ids,
                    train_ids=train_ids)

<!-- %% -->
